In [1]:
import pickle as pkl
import os 
import sys
import numpy as np

import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from torch_geometric.data import Data
sys.path.append("/home/ec2-user/proj/code/graphbert/src")

from utility.prompting import (
    Item,
    get_prompt_tuning_prompt
)



/opt/conda/envs/zheng_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/zheng_env/lib/python3.11/site-packages/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /opt/conda/envs/zheng_env/lib/python3.11/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/opt/conda/envs/zheng_env/lib/python3.11/site-packages/torch_geometric/typing.py:90: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /opt/conda/envs/zheng_env/lib/python3.11/site-packages/torch_spline_conv/_basis_cuda.so)
  warnings.warn(
/opt/conda/envs/z

In [2]:
from tuner import GraphPeftType, GraphPromptTuningConfig
from peft import TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from mapping import get_peft_graph_model
# load a pretrained gpt-2 model
model_name_or_path = "/home/ec2-user/proj/llm_models/vicuna-7b-v1.5"
tokenizer_name_or_path = "/home/ec2-user/proj/llm_models/vicuna-7b-v1.5"
peft_config = GraphPromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    input_embedding_dim=768,
    num_virtual_tokens=4,
    encoder_hidden_size=1024,
    embed_projection=True
)

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.05s/it]
/opt/conda/envs/zheng_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/zheng_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should 

In [4]:
graph_prompt_model = get_peft_graph_model(model, peft_config)
print(graph_prompt_model.print_trainable_parameters())

trainable params: 17,581,056 || all params: 6,755,996,672 || trainable%: 0.26022890261127707
None


In [5]:
graph_prompt_model.prompt_encoder['default'].transform[0].weight

Parameter containing:
tensor([[-0.0024, -0.0086,  0.0236,  ..., -0.0226, -0.0161, -0.0140],
        [-0.0068,  0.0029, -0.0024,  ..., -0.0354,  0.0188, -0.0202],
        [-0.0272,  0.0012,  0.0181,  ..., -0.0106,  0.0248, -0.0328],
        ...,
        [-0.0284, -0.0153,  0.0341,  ...,  0.0267,  0.0100,  0.0213],
        [-0.0059,  0.0313,  0.0216,  ...,  0.0057, -0.0272,  0.0299],
        [ 0.0025,  0.0030,  0.0241,  ..., -0.0223, -0.0080,  0.0173]],
       requires_grad=True)

In [6]:
peft_model_id = '/home/ec2-user/proj/code/graphbert/saved_models/aids-vicuna-7b-v1.5'
graph_prompt_model.load_adapter(peft_model_id, adapter_name='default')

_IncompatibleKeys(missing_keys=['base_model.model.embed_tokens.weight', 'base_model.model.layers.0.self_attn.q_proj.weight', 'base_model.model.layers.0.self_attn.k_proj.weight', 'base_model.model.layers.0.self_attn.v_proj.weight', 'base_model.model.layers.0.self_attn.o_proj.weight', 'base_model.model.layers.0.mlp.gate_proj.weight', 'base_model.model.layers.0.mlp.up_proj.weight', 'base_model.model.layers.0.mlp.down_proj.weight', 'base_model.model.layers.0.input_layernorm.weight', 'base_model.model.layers.0.post_attention_layernorm.weight', 'base_model.model.layers.1.self_attn.q_proj.weight', 'base_model.model.layers.1.self_attn.k_proj.weight', 'base_model.model.layers.1.self_attn.v_proj.weight', 'base_model.model.layers.1.self_attn.o_proj.weight', 'base_model.model.layers.1.mlp.gate_proj.weight', 'base_model.model.layers.1.mlp.up_proj.weight', 'base_model.model.layers.1.mlp.down_proj.weight', 'base_model.model.layers.1.input_layernorm.weight', 'base_model.model.layers.1.post_attention_l

In [7]:
graph_prompt_model.prompt_encoder['default'].transform[0].weight

Parameter containing:
tensor([[ 0.0118,  0.0028,  0.0244,  ..., -0.0306, -0.0337, -0.0269],
        [-0.0220, -0.0094, -0.0330,  ...,  0.0204,  0.0309, -0.0071],
        [-0.0018, -0.0394,  0.0371,  ...,  0.0264,  0.0151, -0.0299],
        ...,
        [ 0.0198, -0.0023, -0.0139,  ...,  0.0207,  0.0065, -0.0338],
        [ 0.0345, -0.0142, -0.0140,  ..., -0.0356,  0.0349, -0.0029],
        [ 0.0105, -0.0056,  0.0206,  ..., -0.0313, -0.0208, -0.0024]],
       requires_grad=True)

: 